In [8]:
id = 0

class DummyClass(object):
    def __init__(self):
        self.id = globals()["id"]
        globals()["id"] += 1

    def __lt__(self, other):
        return self.id < other.id

    def __str__(self):
        return "DummyClass("+str(self.id)+")"

a = DummyClass()
b = DummyClass()

print a, b
print min(a, b)
print min(b, a)
print max(a, b)
print max(b, a)

DummyClass(0) DummyClass(1)
DummyClass(0)
DummyClass(0)
DummyClass(1)
DummyClass(1)
